In [1]:
# %%
import keras
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Dropout, LSTM, Activation, BatchNormalization
from keras import Sequential
import tensorflow as tf
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import pickle
import cv2
import mediapipe as mp
import numpy as np
import os
import copy
import itertools

# %%
dataset = "D:\isl_projects\datasets"




In [13]:
directory = r"D:\isl_projects\datasets"  # replace with your directory path

files = os.listdir(directory)
file_list = []

for file in files:
    file_list.append(file)

print(len(file_list))

262


In [14]:
classList = file_list

print(len(classList))
hands = mp.solutions.hands.Hands(static_image_mode=False, max_num_hands=2,
                                 min_detection_confidence=0.5, min_tracking_confidence=0.5)
sequenceLength = 30

262


In [8]:
def normalizeCoordinates(coords):

    baseX = 0
    baseY = 0

    for i, val in enumerate(coords):
        if i == 0:
            baseX = val[0]
            baseY = val[1]

        coords[i][0] = coords[i][0] - baseX
        coords[i][1] = coords[i][1] - baseY

    coords = list(itertools.chain.from_iterable(coords))

    maxVal = max(list(map(abs, coords)))

    def normalize_(n):
        return n / maxVal

    coords = list(map(normalize_, coords))

    return coords


In [9]:
def skeletonExtraction(path):

    left = []
    right = []
    cap = cv2.VideoCapture(path)
    
    while True:
        success, img = cap.read()
        #cv2.imshow('Mediapipe Feed', img)
        #if cv2.waitKey(10) & 0xFF == ord('q'):
         #   break
        if (success == False):

            cap.release()
            break

        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = hands.process(imgRGB)

        hLeft = None
        hRight = None

        if (results.multi_hand_landmarks):
            for idx, handLms in enumerate(results.multi_hand_landmarks):

                hand = []
                for id, lm in enumerate(handLms.landmark):
                    h, w, c = img.shape
                    cx, cy = int(lm.x * w), int(lm.y*h)
                    hand.append([cx, cy])

                label = results.multi_handedness[idx].classification[0].label

                if (label == 'Left'):
                    hLeft = normalizeCoordinates(hand)
                elif (label == 'Right'):
                    hRight = normalizeCoordinates(hand)

        if (hLeft != None):
            left.append(hLeft)
        if (hRight != None):
            right.append(hRight)

    countLeft = len(left)
    countRight = len(right)
    windowLeft = max(countLeft/sequenceLength, 1)
    windowRight = max(countRight/sequenceLength, 1)

    finalFeatures = []

    if countLeft < sequenceLength or countRight < sequenceLength:
        return []

    for i in range(0, sequenceLength):

        finalFeatures.append(
            left[int(i * windowLeft)] + right[int(i * windowRight)])
    #cap.release()
    #cv2.destroyAllWindows()
    return np.asarray(finalFeatures)

In [17]:
def createDataset():

    features = []
    labels = []
    paths = []

    for index, name in enumerate(classList):
        filesList = os.listdir(os.path.join(dataset, name))

        for i in filesList:

            path = os.path.join(dataset, name, i)

            extractedFeatures = skeletonExtraction(path)

            if (len(extractedFeatures) == sequenceLength):
                features.append(extractedFeatures)
                labels.append(index)
                paths.append(path)

    features = np.asarray(features)
    labels = np.array(labels)

    return features, labels, paths


In [18]:
features, labels, paths = createDataset()

In [19]:
with open('final_features', 'wb') as file:
    # use the dump() method from the pickle module to dump the data into the file
    pickle.dump(features, file)
with open('final_labels', 'wb') as file:
    # use the dump() method from the pickle module to dump the data into the file
    pickle.dump(labels, file)
with open('final_paths', 'wb') as file:
    # use the dump() method from the pickle module to dump the data into the file
    pickle.dump(paths, file)

In [21]:
encodedLabels = to_categorical(labels)

# %%
x_train, x_test, y_train, y_test = train_test_split(
    features, encodedLabels, test_size=0.01, random_state=69)


# %%
x_train.shape

# %%

# %%
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(30, 84)))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(LSTM(256))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

# %%

# %%
earlyStopping = EarlyStopping(
    monitor='val_loss', patience=10, mode='min', restore_best_weights=True)
model.fit(x=x_train, y=y_train, epochs=200, validation_split=0.2)

# %%
modelEvaluate = model.evaluate(x_test, y_test)

# %%
model.save_weights("final_weights.h5")
modelJSON = model.to_json()
with open('final_model.json', 'w') as jsonFile:
    jsonFile.write(modelJSON)

Epoch 1/200
85/85 [==============================] - 16s 126ms/step - loss: 5.4602 - accuracy: 0.0126 - val_loss: 5.4156 - val_accuracy: 0.0281
Epoch 2/200
85/85 [==============================] - 11s 133ms/step - loss: 4.6682 - accuracy: 0.0545 - val_loss: 4.9780 - val_accuracy: 0.0637
Epoch 3/200
85/85 [==============================] - 13s 156ms/step - loss: 3.9961 - accuracy: 0.1112 - val_loss: 4.2906 - val_accuracy: 0.1496
Epoch 4/200
85/85 [==============================] - 11s 128ms/step - loss: 3.4328 - accuracy: 0.1908 - val_loss: 3.5534 - val_accuracy: 0.1881
Epoch 5/200
85/85 [==============================] - 13s 156ms/step - loss: 2.9651 - accuracy: 0.2616 - val_loss: 3.0082 - val_accuracy: 0.2504
Epoch 6/200
85/85 [==============================] - 13s 156ms/step - loss: 2.5829 - accuracy: 0.3361 - val_loss: 2.6379 - val_accuracy: 0.3289
Epoch 7/200
85/85 [==============================] - 13s 156ms/step - loss: 2.2656 - accuracy: 0.3827 - val_loss: 2.1546 - val_accuracy:

85/85 [==============================] - 13s 158ms/step - loss: 0.0511 - accuracy: 0.9881 - val_loss: 0.9360 - val_accuracy: 0.8504
Epoch 114/200
85/85 [==============================] - 14s 159ms/step - loss: 0.0522 - accuracy: 0.9874 - val_loss: 0.9569 - val_accuracy: 0.8400
Epoch 115/200
85/85 [==============================] - 14s 166ms/step - loss: 0.0389 - accuracy: 0.9874 - val_loss: 0.9777 - val_accuracy: 0.8489
Epoch 116/200
85/85 [==============================] - 14s 165ms/step - loss: 0.0563 - accuracy: 0.9841 - val_loss: 1.0769 - val_accuracy: 0.8311
Epoch 117/200
85/85 [==============================] - 13s 158ms/step - loss: 0.1778 - accuracy: 0.9533 - val_loss: 1.2885 - val_accuracy: 0.7926
Epoch 118/200
85/85 [==============================] - 13s 154ms/step - loss: 0.1109 - accuracy: 0.9674 - val_loss: 1.0741 - val_accuracy: 0.8237
Epoch 119/200
85/85 [==============================] - 13s 155ms/step - loss: 0.0610 - accuracy: 0.9815 - val_loss: 1.0762 - val_accuracy:

In [10]:
def inputProcessing(path):
    features = skeletonExtraction(path)
    temp = [features]

    return np.asarray(temp)

In [11]:
jsonFile = open('final_model.json', 'r')
loadedModel = jsonFile.read()
loadedModel = keras.models.model_from_json(loadedModel)
loadedModel.load_weights('final_weights.h5')
loadedModel.compile(loss='categorical_crossentropy',
                    optimizer='adam', metrics=['accuracy'])

In [15]:
features = inputProcessing(r"C:\Users\91790\Desktop\1682510507.0836751video.avi")
output = loadedModel.predict(features)
label = classList[np.argmax(output)]

1/1 [==============================] - 1s 1s/step


In [16]:
label

'Good Morning'

In [ ]:
from sklearn.metrics import precision_recall_curve, roc_curve, accuracy_score, confusion_matrix,
precision_score, recall_score

In [3]:
loadedModel.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 30, 128)           109056    
                                                                 
 activation_3 (Activation)   (None, 30, 128)           0         
                                                                 
 dropout_3 (Dropout)         (None, 30, 128)           0         
                                                                 
 lstm_3 (LSTM)               (None, 256)               394240    
                                                                 
 activation_4 (Activation)   (None, 256)               0         
                                                                 
 dropout_4 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)              